In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import *

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

In [8]:
graph_schema = StructType([
    StructField("to", IntegerType(), False),
    StructField("from", IntegerType(), False)
])

In [9]:
dist_schema = StructType([
    StructField("vertex", IntegerType(), False),
    StructField("distance", IntegerType(), False)
])

In [70]:
def shortest_path(v_from, v_to, dataset_path=None):
    edges = spark.read.csv(dataset_path, sep="\t", schema=graph_schema).repartition(60)
    distances = spark.createDataFrame([[v_from,int(0)]],schema=dist_schema)
    path_length = 0
    while True:
        new_distances=distances.join(edges,col("from")==col("vertex")).select(col("to").alias("vertex"),distances.distance)
              
               
        cnt=new_distances.filter(edges.to==v_to).count()
        if cnt==0:
            distances=new_distances.selectExpr("vertex","distance+1 as distance ")
        else:
            break
    
    # Here you should implement the BFS algorithm. It should return the length
    # of the minimal path (single integer) between v_from and v_to
    path_length=new_distances.agg({"distance": "min"}).collect()[0][0]
    if path_length>0:
        path_length+=1
    return path_length

In [71]:
print shortest_path(12, 34, "/data/twitter/twitter_sample.txt")

8
